# Climate DKG and variable annotations

In [9]:
import requests
import pandas
from mira.sources.acsets.decapodes import process_decapode
from mira.dkg.askemo import get_askem_climate_ontology_terms
from mira.dkg.physical_constants import get_physical_constant_terms

### Extract synonym mappings for the Halfar model

In [16]:
synonym_mappings = {}

for wd, name, description, synonyms, xrefs, _value, _formula, symbols in get_physical_constant_terms():
    curie = f"wikidata:{wd}"
    synonym_mappings[name] = curie
    for synoynm in synonyms or []:
        synonym_mappings[synoynm] = (curie, name)
    for symbol in symbols or []:
        synonym_mappings[symbol] = (curie, name)

for curie, term in get_askem_climate_ontology_terms().items():
    if "askem.climate:0001002" in term.part_ofs:
        print(curie, term.name, term.synonyms)
        for syn in term.synonyms:
            if syn.type == 'referenced_by_symbol':
                synonym_mappings[syn.value] = (term.id, term.name)

askem.climate:0000102 gravitational constant [Synonym(value='g', type='referenced_by_symbol')]
askem.climate:0002005 flow law of ice []
askem.climate:0002006 Glen's law []
askem.climate:0003012 density [Synonym(value='ρ', type='referenced_by_symbol')]
askem.climate:0003018 Ice height [Synonym(value='h', type='referenced_by_symbol')]
askem.climate:0003019 Strain rate [Synonym(value='Γ', type='referenced_by_symbol')]
askem.climate:0003020 Glen flow law exponent [Synonym(value='n', type='referenced_by_symbol')]
askem.climate:0003021 Power law constant [Synonym(value='A', type='referenced_by_symbol')]


### Create a Decapodes object from the Halfar model JSON

In [17]:
model_url = 'https://raw.githubusercontent.com/ciemss/Decapodes.jl/sa_climate_modeling/examples/climate/ice_dynamics.json'
model_json = requests.get(model_url).json()
model_decapode = process_decapode(model_json)

### Look up groundings for model variables and show the ones that exist

In [18]:
for var in model_decapode.variables.values():
    if var.name in synonym_mappings:
        curie, name = synonym_mappings[var.name]
        prefix, identifier = curie.split(':', 1)
        var.identifiers = {prefix: identifier}
        print(var.name, var.identifiers, name)

h {'askem.climate': '0003018'} Ice height
Γ {'askem.climate': '0003019'} Strain rate
n {'askem.climate': '0003020'} Glen flow law exponent


### General physical constants in the DKG

In [42]:
physical_constants = {}
for wd, name, description, synonyms, xrefs, _value, _formula, symbols in get_physical_constant_terms():
    curie = f"wikidata:{wd}"
    physical_constants[name] = \
        {
            'curie': curie,
            'description': description,
            'synonyms': ', '.join(synonyms),
            'xrefs': ', '.join(xrefs),
            'value': _value,
            'symbols': ', '.join(symbols)
        }

In [56]:
pandas.set_option('display.max_colwidth', None)
def get_df(constant_name):
    return pandas.DataFrame({k: [v] for k, v in physical_constants[constant_name].items()}).T

In [57]:
get_df('Planck constant')

,0
curie,wikidata:Q122894
description,physical constant representing the quantum of action
synonyms,"h, 𝒉, Planck's constant"
xrefs,"goldbook:P04685, nist.codata:h"
value,0.000000000000000000000000000000000662607015
symbols,h


In [58]:
get_df('electron mass')

,0
curie,wikidata:Q3814108
description,mass of a stationary electron
synonyms,electron rest mass
xrefs,"goldbook:E02008, nist.codata:me"
value,0.00000000000000000000000000000091093837015
symbols,m_{\mathrm {e} }
